# Mount google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install packages

In [2]:
pip install assemblyai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.6/72.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [3]:
pip install praat-textgrids

In [4]:
pip install pydub

In [5]:
pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 29.2 MB/s eta 0:00:00


# Experiment label

These should be the parent and subfolder names on Google Drive

In [6]:
exp_label = "PPP3"
exp_spec = "prolificpilot3"

# Functions

In [7]:
# transcribe wav files and save transcriptions in a dictionary
def transcribe(api_key, filenames, wav_dir, output_dictionary_dir, language_code):
    import os
    import assemblyai as aai
    import pickle

    # your AssemblyAI API key
    aai.settings.api_key = api_key

    # set language
    config = aai.TranscriptionConfig(language_code=language_code)

    # create transcriber
    transcriber = aai.Transcriber(config=config)

    # initialize transcription dictionary
    transc_dict = {}

    for file in filenames:
        # load wav file
        filepath = os.path.join(wav_dir, file)

        # transcribe wav file
        transcript = transcriber.transcribe(filepath)

        if transcript.error:
            print(file, transcript.error)
            transc_dict[file] = 'not_recognized'
        else:
            # save transcription in dictionary
            transc_dict[file] = transcript.text  or 'not_recognized'
            print(f"{file}: {transcript.text}")

    # save the dictionary
    with open(os.path.join(output_dictionary_dir, "transcriptions.p"), "wb") as f:
        pickle.dump(transc_dict, f)

    # format final transcriptions
    final_transc = {os.path.splitext(key)[0]: val for key, val in transc_dict.items()}

    return final_transc

In [8]:
# This function is adapted from: https://github.com/Masato-Nakamura-3/TransCloze
def chronset_results_to_dictionary(chronset_dir, output_onset_dir):
    import os
    import pandas as pd
    import pickle

    onset_dict = dict()
    for textf in [t for t in os.listdir(chronset_dir) if ".txt" in t]:
        onset_df = pd.read_table(os.path.join(chronset_dir, textf), names = ["wav", "rt"])
        temp_dict = dict(zip([os.path.splitext(w)[0] for w in onset_df["wav"]], onset_df["rt"] / 1000))
        onset_dict.update(temp_dict)

    # save the dictionary
    with open(os.path.join(output_onset_dir, "onset.p"), "wb") as f:
        pickle.dump(onset_dict, f)

    return onset_dict

In [9]:
# This function is adapted from: https://github.com/Masato-Nakamura-3/TransCloze
# This function generates .TextGrid files for Praat
# You can optionally pass a dictionary for the onset and/or for the duration of the files
# The onset function is basically for Chronset
# You can also give it context info to help hand correction

# If the audio files have heterogeneous durations, add an argument of "inconsistent = 1"

# The function replaces the RT with 0.1 if the RT is 0 or shorter, or is NaN, producing an error message

def create_textgrids(dict_transcription, output_dir, wav_dir, onset = dict(), inconsistent = False):
    import os
    import textgrids
    import numpy as np
    from pydub import AudioSegment

    filenames = [os.path.splitext(i)[0] for i in os.listdir(wav_dir) if (".wav" in i)]

    for k in filenames:
        # Load the TextGrid template and edit it
        tg = textgrids.TextGrid(f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw/temp_note.TextGrid")

        # If the duration of the audio files are inconsistent, edit each TextGrid files to match the duration of the audio files
        if inconsistent == True:
            audio = AudioSegment.from_file(os.path.join(wav_dir, k + ".wav"))
            tg["words"][1].xmax = tg["words"][-1].xmin = audio.duration_seconds - 0.001

            if tg.xmax != audio.duration_seconds:
                tg.xmax = audio.duration_seconds

                for tier in tg.keys():
                    tg[tier][-1].xmax = audio.duration_seconds

        tg["words"][1].text = dict_transcription[k + ".wav"]

        if k in onset.keys():
            # If the onset is not detected or is negative, set 0.1
            if np.isnan(onset[k]) or onset[k] <= 0 :
                print("No onset for ", k)
                tg["words"][0].xmax = tg["words"][1].xmin = 0.1
            else:
                tg["words"][0].xmax = tg["words"][1].xmin = onset[k]

        item_id = k.split("_")[1]

        tg.write(os.path.join(output_dir, k + ".TextGrid"))

    print("All textgrid files generated.")

# Designate file paths

In [10]:
# This pipeline uses automatic speech-to-text software provided by https://www.assemblyai.com/
# Create an account and replace the code below with your personal API key.
api_key = 'd424d88155ec405cafa0811227e73c33'

In [11]:
# path to folder containing wav files
wav_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav'

# get list of wav file names
import os
filenames = [w for w in os.listdir(wav_dir) if (".wav" in w)]

# path to output folder with transcription dictionary
output_dictionary_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/transc_output'

# path to zip folder with wav files for chronset
chronset_input_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/chronset_input'

# path to output folder with onset
chronset_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/chronset_results'

# path to output folder with onset
output_onset_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/onset_output'

# path to output folder with textgrids (with transcriptions)
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'

# Run automatic transcriptions

In [ ]:
import os
len(os.listdir(wav_dir))

1291

In [ ]:
# generate transcriptions and store them in a dictionary
# replace "language_code" with your language code (e.g., American English : "en_us"); check language_code here: https://www.assemblyai.com/docs/getting-started/supported-languages
language_code = "en_us"
dict_transcription = transcribe(api_key, filenames, wav_dir, output_dictionary_dir, language_code)

In [ ]:
len(dict_transcription)

# Run script below if you only need speech onset data (get onset data using Chronset)

This section uses Chronset to get onset data and combines with with transcriptions

In [ ]:
import os
import zipfile
from pydub import AudioSegment
import speech_recognition as sr

# Set the name of the zip files
# The output files will be named as "{experiment_name}_{number}".zip (e.g. planet_cloze_1.zip)
zipname = f"{exp_label}_{exp_spec}_"

# # Check wav files for errors
# for f in filenames:
#     fsize = os.path.getsize(os.path.join(wav_dir, f))
#     if fsize < 100000:
#         print(f)
#         print(fsize)
#         with sr.AudioFile(os.path.join(wav_dir, f)) as wavf:
#             print(wavf.DURATION)

# Create batches of 500 wav files (i.e., maximum file counts Chronset can handle)
# Make lists of 500 or fewer wav files
fewerthan500 = []
temp = []
for f in filenames:
    temp.append(f)
    if len(temp) == 500:
        fewerthan500.append(temp)
        temp = []
if len(temp) > 0:
    fewerthan500.append(temp)

# Total nuber of wav files
len(filenames)
# Number of zip files
len(fewerthan500)

# Output zip files (with 500 or fewer wav files)
fnum = 0
for filelist in fewerthan500:
    fnum = fnum + 1
    with zipfile.ZipFile(os.path.join(chronset_input_dir, zipname+str(fnum) + ".zip"), 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
        for f in filelist:
            new_zip.write(os.path.join(wav_dir, f), arcname = f)

Go to https://www.bcbl.eu/databases/chronset and upload each zip file that is in the **chronset_input** folder.

Retrieve Chronset results from your mailbox and upload those files to the **chronset_results** folder.

In [ ]:
# get onsets and store them in a dictionary
import pickle
onset = chronset_results_to_dictionary(chronset_dir, output_onset_dir)
onset = pickle.load( open( f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/onset_output/onset.p", "rb" ) )

In [ ]:
# generate textgrid files and store them in tg_dir_final
import pickle
transcriptions = pickle.load( open( f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/transc_output/transcriptions.p", "rb" ) )
create_textgrids(transcriptions, tg_dir, wav_dir, onset, inconsistent = True)

No onset for  28_SP_bcob
No onset for  33_PP_hbvp
No onset for  36_SS_hbvp
No onset for  38_SP_kojh
No onset for  44_SS_hbvp
No onset for  4_masculine_mismatch_bcob
No onset for  58_unergative_bcob
No onset for  59_unergative_bcob
No onset for  64_unergative_bcob
No onset for  77_unaccusative_hbvp
No onset for  80_unaccusative_bcob
No onset for  7_masculine_mismatch_nzne
All textgrid files generated.


# Run script below if you need onset + alignment of multi-word utterances

This script does not produce onset data, only transcriptions.

In [ ]:
# generate textgrid files and store them in tg_dir_final
import pickle
transcriptions = pickle.load( open( f"/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/transc_output/transcriptions.p", "rb" ) )
create_textgrids(transcriptions, tg_dir, wav_dir, inconsistent = True)

All textgrid files generated.


In [ ]:
len(transcriptions)

1290

In [ ]:
import os
len(os.listdir(tg_dir))

1291

# Prepare files for running MFA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
len(os.listdir(tg_dir))

1291

In [ ]:
# Define the source folder paths
wav_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav'
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
wav_tg_comb = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav_tg_comb'

# Create the destination folder if it doesn't exist
if not os.path.exists(wav_tg_comb):
    os.makedirs(wav_tg_comb)

def copy_folder_contents(source, destination):
    import shutil
    for item in os.listdir(source):
        source_item = os.path.join(source, item)
        destination_item = os.path.join(destination, item)
        # If it's a file, copy it
        if os.path.isfile(source_item):
            shutil.copy(source_item, destination_item)
        # If it's a folder, recursively copy its contents
        elif os.path.isdir(source_item):
            if not os.path.exists(destination_item):
                os.makedirs(destination_item)
            copy_folder_contents(source_item, destination_item)

copy_folder_contents(wav_dir, wav_tg_comb)
copy_folder_contents(tg_dir, wav_tg_comb)

print("Files have been successfully copied to the new folder.")

Files have been successfully copied to the new folder.


In [ ]:
wav_tg_comb_backup = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav_tg_comb_backup'
if not os.path.exists(wav_tg_comb_backup):
    os.makedirs(wav_tg_comb_backup)
copy_folder_contents(wav_tg_comb, wav_tg_comb_backup)
print("Folder has been successfully duplicated.")

Folder has been successfully duplicated.


In [ ]:
len(os.listdir(wav_dir)), len(os.listdir(tg_dir)), len(os.listdir(wav_tg_comb), len(os.listdir(wav_tg_comb_backup))

2582

In [ ]:
# import wave
# def check_wav_channels(wav_file_path):
#     with wave.open(wav_file_path, 'r') as wav_file:
#         channels = wav_file.getnchannels()
#         if channels == 1:
#             return "Mono"
#         elif channels == 2:
#             return "Stereo"
#         else:
#             return f"Multi-channel ({channels} channels)"

In [ ]:
import os
from shutil import move

# Set your source directory
wav_tg_comb = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav_tg_comb'
files_per_subfolder = 100  # Adjust the number of pairs you want per subfolder

# List all files and filter out non '.wav' and '.textgrid' files
all_files = os.listdir(wav_tg_comb)
wav_files = sorted([f for f in all_files if f.endswith('.wav')])
textgrid_files = sorted([f for f in all_files if f.endswith('.TextGrid')])

# Combine the files into pairs based on matching base filenames
pairs = []
for wav_file in wav_files:
    base_name = os.path.splitext(wav_file)[0]
    textgrid_file = base_name + '.TextGrid'
    if textgrid_file in textgrid_files:
        pairs.append((wav_file, textgrid_file))

In [ ]:
# Function to create subfolders and move files
def create_subfolders_and_move_files(pairs, source_directory, files_per_subfolder):
    subfolder_count = 1
    for i in range(0, len(pairs), files_per_subfolder):
        # Create a new subfolder in the source directory
        subfolder_path = os.path.join(source_directory, f'subfolder_{subfolder_count}')
        os.makedirs(subfolder_path, exist_ok=True)
        # Move a batch of pairs to the new subfolder
        for wav_file, textgrid_file in pairs[i:i + files_per_subfolder]:
            move(os.path.join(source_directory, wav_file), os.path.join(subfolder_path, wav_file))
            move(os.path.join(source_directory, textgrid_file), os.path.join(subfolder_path, textgrid_file))
        subfolder_count += 1
    print("Files have been successfully divided into subfolders.")

# Call the function
create_subfolders_and_move_files(pairs, wav_tg_comb, files_per_subfolder)

Files have been successfully divided into subfolders.


# RUN MFA

On your local computer, inside MFA folder (e.g., ~/Documents/MFA):
*   Create new folder and name it using the same format as in Google Drive (e.g., **PPP3_pilot**)
*   Upload **run_mfa_on_subsets.sh** file (from Github pipeline)

Inside the experiment folder (e.g., ~/Documents/MFA/PPP3_pilot):
*   Upload **wav_tg_comb** folder (from Google Drive)
*   Create new folder and name it **MFA_output**

Open **run_mfa_on_subsets.sh** file with text editor:
*   Change file paths to your experiment folders, save and close

Open terminal and run each code below line by line:
*   conda activate aligner
*   cd ~/Documents/MFA
*   chmod +x run_mfa_on_subsets.sh
*   ./run_mfa_on_subsets.sh

Upload **MFA_output** folder to your experiment folder on Google Drive


# Combine files after running MFA

Upload MFA-processed results folder to Google Drive and rename it: MFA_processed

In [ ]:
parent_directory = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/MFA_output'
# parent_directory = f'/Users/rosalee/Documents/MFA/{exp_label}/{exp_spec}_output'

destination_directory = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'
# destination_directory = f'/Users/rosalee/Documents/MFA/{exp_label}/{exp_spec}_output_comb'

import os
os.makedirs(destination_directory, exist_ok=True)  # Create the destination directory if it doesn't exist

def copy_textgrid_files(source_dir, dest_dir):
    from shutil import copy2
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.TextGrid'):
                source_file_path = os.path.join(root, file)
                destination_file_path = os.path.join(dest_dir, file)

                # Check if a file with the same name already exists in the destination directory
                if os.path.exists(destination_file_path):
                    # If so, create a new file name to avoid overwriting
                    base, extension = os.path.splitext(file)
                    counter = 1
                    new_file_name = f"{base}_{counter}{extension}"
                    destination_file_path = os.path.join(dest_dir, new_file_name)

                    # Increment the counter until a unique name is found
                    while os.path.exists(destination_file_path):
                        counter += 1
                        new_file_name = f"{base}_{counter}{extension}"
                        destination_file_path = os.path.join(dest_dir, new_file_name)

                # Copy the file
                copy2(source_file_path, destination_file_path)

    print("All TextGrid files have been copied to the destination folder: tg_MFA_processed.")

# Call the function
copy_textgrid_files(parent_directory, destination_directory)

All TextGrid files have been copied to the destination folder: tg_MFA_processed.


In [ ]:
# Check number of tg files in pre-MFA (tg_raw) & post-MFA (tg_MFA_processed) folders
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'

len(os.listdir(tg_dir)), len(os.listdir(tg_MFA_processed_dir))

(1291, 1204)

In [ ]:
# import os

# def list_files_not_in_other_folder(source_dir, comparison_dir):
#     source_files = set(os.listdir(source_dir))
#     comparison_files = set(os.listdir(comparison_dir))

#     # Find files that are in source_dir but not in comparison_dir
#     files_not_in_comparison = source_files - comparison_files

#     if files_not_in_comparison:
#         print("Files in source but not in comparison directory:")
#         for file in files_not_in_comparison:
#             print(file)
#     else:
#         print("All files are present in both directories.")

# tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
# tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'

# list_files_not_in_other_folder(tg_dir, tg_MFA_processed_dir)

In [ ]:
import os
import shutil

def copy_files_not_in_other_folder(source_dir, comparison_dir, new_folder):
    # Create the new folder if it doesn't exist
    if not os.path.exists(new_folder):
        os.makedirs(new_folder)

    source_files = set(os.listdir(source_dir))
    comparison_files = set(os.listdir(comparison_dir))

    # Find files that are in source_dir but not in comparison_dir
    files_not_in_comparison = source_files - comparison_files

    if files_not_in_comparison:
        print("Copying files from source to new folder:")
        for file in files_not_in_comparison:
            source_file_path = os.path.join(source_dir, file)
            new_file_path = os.path.join(new_folder, file)
            # Move (or copy) the file to the new folder
            shutil.copy(source_file_path, new_file_path)
            print(f"Copied: {file}")
    else:
        print("All files are present in both directories, no files to move.")

tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'
new_folder = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_to_process'

copy_files_not_in_other_folder(tg_dir, tg_MFA_processed_dir, new_folder)

In [ ]:
len(os.listdir(new_folder))

173

In [ ]:
import os
import shutil

def copy_matching_wav_files(wav_dir, textgrid_folder, new_wav_folder):
    # Create the new folder if it doesn't exist
    if not os.path.exists(new_wav_folder):
        os.makedirs(new_wav_folder)

    # Get the list of .TextGrid files (without extensions) in the textgrid_folder
    textgrid_files = {os.path.splitext(file)[0] for file in os.listdir(textgrid_folder) if file.endswith('.TextGrid')}

    # Get all .wav files from wav_dir and copy those that match the .TextGrid file names
    wav_files = [file for file in os.listdir(wav_dir) if file.endswith('.wav')]

    for wav_file in wav_files:
        wav_file_base = os.path.splitext(wav_file)[0]  # Get the name without extension
        if wav_file_base in textgrid_files:
            source_wav_path = os.path.join(wav_dir, wav_file)
            destination_wav_path = os.path.join(new_wav_folder, wav_file)
            shutil.copy(source_wav_path, destination_wav_path)
            print(f"Copied: {wav_file}")

    if not textgrid_files:
        print("No matching .wav files to copy.")

# Example usage
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'
new_folder = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_to_process'
wav_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav'
new_wav_folder = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav_to_process'

# Copy matching wav files
copy_matching_wav_files(wav_dir, new_folder, new_wav_folder)


In [ ]:
import os
import shutil

def copy_matching_wav_files(wav_dir, textgrid_folder):
    # Get the list of .TextGrid files (without extensions) in the textgrid_folder
    textgrid_files = {os.path.splitext(file)[0] for file in os.listdir(textgrid_folder) if file.endswith('.TextGrid')}

    # Get all .wav files from wav_dir and copy those that match the .TextGrid file names
    wav_files = [file for file in os.listdir(wav_dir) if file.endswith('.wav')]

    for wav_file in wav_files:
        wav_file_base = os.path.splitext(wav_file)[0]  # Get the name without extension
        if wav_file_base in textgrid_files:
            source_wav_path = os.path.join(wav_dir, wav_file)
            destination_wav_path = os.path.join(textgrid_folder, wav_file)  # Copy to textgrid_folder
            shutil.copy(source_wav_path, destination_wav_path)
            print(f"Copied: {wav_file}")

    if not textgrid_files:
        print("No matching .wav files to copy.")

# Example usage
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'
new_folder = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_to_process'
wav_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/wav'

# Copy matching wav files into the same folder as the TextGrid files
copy_matching_wav_files(wav_dir, new_folder)

Copied: 28_SP_bcob.wav
Copied: 29_SS_bcob.wav
Copied: 30_PP_bcob.wav
Copied: 2_feminine_mismatch_bcob.wav
Copied: 31_PS_bcob.wav
Copied: 32_SP_bcob.wav
Copied: 34_PP_bcob.wav
Copied: 33_SS_bcob.wav
Copied: 35_PS_bcob.wav
Copied: 36_SP_bcob.wav
Copied: 37_SS_bcob.wav
Copied: 38_SP_kojh.wav
Copied: 37_SS_rror.wav
Copied: 38_PP_rror.wav
Copied: 38_SP_jmgf.wav
Copied: 38_PP_bcob.wav
Copied: 3_feminine_match_bcob.wav
Copied: 39_PS_bcob.wav
Copied: 40_SP_bcob.wav
Copied: 42_PP_bcob.wav
Copied: 41_SS_bcob.wav
Copied: 43_SS_itng.wav
Copied: 43_PS_bcob.wav
Copied: 44_SP_bcob.wav
Copied: 46_PP_bcob.wav
Copied: 45_SS_bcob.wav
Copied: 48_SP_bcob.wav
Copied: 47_PS_bcob.wav
Copied: 49_SS_bcob.wav
Copied: 50_PP_bcob.wav
Copied: 4_masculine_mismatch_bcob.wav
Copied: 51_PS_bcob.wav
Copied: 52_SP_bcob.wav
Copied: 53_SS_bcob.wav
Copied: 54_PP_bcob.wav
Copied: 55_PS_bcob.wav
Copied: 57_unergative_bcob.wav
Copied: 56_SP_bcob.wav
Copied: 58_unergative_bcob.wav
Copied: 59_unergative_bcob.wav
Copied: 5_mascul

In [ ]:
len(os.listdir(new_folder))/2

86.5

In [ ]:
# # Function to copy files from tg_raw if they don't exist in MFA_output
# def copy_files_if_not_exists(source, destination):
#     import shutil
#     import os
#     for filename in os.listdir(source):
#         source_file = os.path.join(source, filename)
#         destination_file = os.path.join(destination, filename)
#         # Check if it's a file and if it doesn't already exist in MFA_output
#         if os.path.isfile(source_file):
#             if not os.path.exists(destination_file):
#                 shutil.copy(source_file, destination_file)
#                 print(f"Copied: {filename}")

# tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
# tg_MFA_processed = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'
# copy_files_if_not_exists(tg_dir, tg_MFA_processed)

# print("Copy operation completed.")

In [ ]:
# Check number of tg files in pre-MFA (tg_raw) & post-MFA (tg_MFA_processed) folders
tg_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_raw'
tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'

len(os.listdir(tg_dir)), len(os.listdir(tg_MFA_processed_dir))

(1291, 1204)

# Generate results file

In [ ]:
import os
import io
import textgrids
import pandas as pd
import numpy as np

tg_MFA_processed_dir = f'/content/drive/MyDrive/{exp_label}/{exp_spec}/FastTranscribeAlign/tg_MFA_processed'

results = []
for tg_loc in [f for f in os.listdir(tg_MFA_processed_dir) if f.endswith(".TextGrid")]:
    tg_path = os.path.join(tg_MFA_processed_dir, tg_loc)
    filename = tg_loc.rsplit(".", 1)[0]
    item = filename.split("_")[0]
    item_type = "_".join(filename.split("_")[1:-1])
    subject_id = filename.split("_")[-1]

    try:
        tg = textgrids.TextGrid(tg_path)
        for word in tg['words']:
            if word.text.strip():
                results.append([filename, item, item_type, subject_id, word.text, word.xmin, word.xmax, word.dur])
    except Exception as e:
        print(f"Skipping file {tg_loc} due to error: {e}")
        # Insert a row indicating the file was not recognized due to an error
        results.append([filename, item, item_type, subject_id, "not_recognized", None, None, None])

df = pd.DataFrame(results, columns = ['filename', 'item', 'item_type', 'subject_id', 'word', 'rt_onset', 'rt_offset', 'rt_dur'])
df[['type', 'condition']] = pd.DataFrame(df['item_type'].apply(lambda x: x.split('_') if '_' in x else [x, None]).tolist(), index=df.index)
df['word'] = df['word'].apply(lambda x: x.strip().lower())
df['item'] = df['item'].astype(int)
df['word_order'] = df.groupby('filename')['word'].cumcount() + 1

df = df.reset_index(drop=True)
df

filename  item          item_type subject_id       word  \
0     6_feminine_mismatch_yjml     6  feminine_mismatch       yjml       lisa   
1     6_feminine_mismatch_yjml     6  feminine_mismatch       yjml     gently   
2     6_feminine_mismatch_yjml     6  feminine_mismatch       yjml  comforted   
3     6_feminine_mismatch_yjml     6  feminine_mismatch       yjml        her   
4     6_feminine_mismatch_yjml     6  feminine_mismatch       yjml        son   
...                        ...   ...                ...        ...        ...   
8867      69_unaccusative_kfyi    69       unaccusative       kfyi    karen's   
8868      69_unaccusative_kfyi    69       unaccusative       kfyi      shirt   
8869      69_unaccusative_kfyi    69       unaccusative       kfyi     didn't   
8870      69_unaccusative_kfyi    69       unaccusative       kfyi        dry   
8871      69_unaccusative_kfyi    69       unaccusative       kfyi   properly   

      rt_onset  rt_offset  rt_dur          type condition  word_order  
0       3.0795     3.4395    0.36      feminine  mismatch           1  
1       3.4395     3.9295    0.49      feminine  mismatch           2  
2       3.9295     4.7395    0.81      feminine  mismatch           3  
3       4.7395     4.9395    0.20      feminine  mismatch           4  
4       4.9395     5.3195    0.38      feminine  mismatch           5  
...        ...        ...     ...           ...       ...         ...  
8867    0.8095     1.1795    0.37  unaccusative      None           1  
8868    1.1795     1.4595    0.28  unaccusative      None           2  
8869    1.4595     1.7595    0.30  unaccusative      None           3  
8870    1.7595     1.9795    0.22  unaccusative      None           4  
8871    1.9795     2.5495    0.57  unaccusative      None           5  

[8872 rows x 11 columns]

In [ ]:
# add attention check column
# df_stim_critical['attention_check'] = 1
df.subject_id.nunique()

16

In [ ]:
PCIbex_results = pd.read_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/{exp_label}_PCIbex_results_{exp_spec}.csv')
PCIbex_results.rename(columns = {'#':'time_id', 'Unnamed: 5':'condition_general', 'Unnamed: 7':'element_type', 'Unnamed: 10':'filename', 'Unnamed: 12':'group', 'Unnamed: 13':'subject_id', 'Unnamed: 14':'item_num', 'Unnamed: 15':'condition','Unnamed: 16':'response'}, inplace = True)
PCIbex_results_MedRec = PCIbex_results[PCIbex_results["element_type"].str.contains("MediaRecorder") == True]
PCIbex_results_id = PCIbex_results_MedRec[["time_id","subject_id"]]
PCIbex_results_id.subject_id.nunique()

16

In [ ]:
#get tg_results file to merge with time id
df_results_id = pd.merge(df, PCIbex_results_id[['time_id','subject_id']].drop_duplicates(), how='right', on=['subject_id'])
df_results_id.subject_id.nunique()

16

In [ ]:
df_results_id = df_results_id.dropna(subset=['filename'])
df_results_id.subject_id.nunique()

15

In [ ]:
# select critical items and combine target verb from stim files

df_crit_results = df_results_id[df_results_id.item < 25]

stim_active = pd.read_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/exp3_stimuli_critical_real.csv')
stim_active = stim_active.rename(columns={'possessor_gender':'type'})

df_crit = pd.merge(df_crit_results,stim_active[['item', 'type','condition','possessor','adverb','verb','pronoun','possessee']],how='left',on=['item', 'type','condition'])

df_crit.possessor = df_crit.possessor.str.lower()
df_crit

filename  item          item_type subject_id       word  \
0     6_feminine_mismatch_yjml   6.0  feminine_mismatch       yjml       lisa   
1     6_feminine_mismatch_yjml   6.0  feminine_mismatch       yjml     gently   
2     6_feminine_mismatch_yjml   6.0  feminine_mismatch       yjml  comforted   
3     6_feminine_mismatch_yjml   6.0  feminine_mismatch       yjml        her   
4     6_feminine_mismatch_yjml   6.0  feminine_mismatch       yjml        son   
...                        ...   ...                ...        ...        ...   
2800     6_feminine_match_nzne   6.0     feminine_match       nzne        her   
2801     6_feminine_match_nzne   6.0     feminine_match       nzne   daughter   
2802     6_feminine_match_nzne   6.0     feminine_match       nzne         at   
2803     6_feminine_match_nzne   6.0     feminine_match       nzne        the   
2804     6_feminine_match_nzne   6.0     feminine_match       nzne      movie   

      rt_onset  rt_offset  rt_dur      type condition  word_order     time_id  \
0       3.0795     3.4395  0.3600  feminine  mismatch         1.0  1727711888   
1       3.4395     3.9295  0.4900  feminine  mismatch         2.0  1727711888   
2       3.9295     4.7395  0.8100  feminine  mismatch         3.0  1727711888   
3       4.7395     4.9395  0.2000  feminine  mismatch         4.0  1727711888   
4       4.9395     5.3195  0.3800  feminine  mismatch         5.0  1727711888   
...        ...        ...     ...       ...       ...         ...         ...   
2800    1.6495     1.7395  0.0900  feminine     match         3.0  1727727799   
2801    1.7395     2.0695  0.3300  feminine     match         4.0  1727727799   
2802    2.0695     2.1895  0.1200  feminine     match         5.0  1727727799   
2803    2.1895     2.2395  0.0500  feminine     match         6.0  1727727799   
2804    2.2395     2.5800  0.3405  feminine     match         7.0  1727727799   

     possessor  adverb       verb pronoun possessee  
0         lisa  gently  comforted     her       son  
1         lisa  gently  comforted     her       son  
2         lisa  gently  comforted     her       son  
3         lisa  gently  comforted     her       son  
4         lisa  gently  comforted     her       son  
...        ...     ...        ...     ...       ...  
2800      lisa  gently  comforted     her  daughter  
2801      lisa  gently  comforted     her  daughter  
2802      lisa  gently  comforted     her  daughter  
2803      lisa  gently  comforted     her  daughter  
2804      lisa  gently  comforted     her  daughter  

[2805 rows x 17 columns]

In [ ]:
def find_rt_values(trial):

    possessor_onset = trial.loc[trial['word'] == trial['possessor'].iloc[0], 'rt_onset'].iloc[0] if not trial.loc[trial['word'] == trial['possessor'].iloc[0], 'rt_onset'].empty else None
    trial['possessor_onset'] = possessor_onset
    possessor_offset = trial.loc[trial['word'] == trial['possessor'].iloc[0], 'rt_offset'].iloc[0] if not trial.loc[trial['word'] == trial['possessor'].iloc[0], 'rt_offset'].empty else None
    trial['possessor_offset'] = possessor_offset

    adverb_onset = trial.loc[trial['word'] == trial['adverb'].iloc[0], 'rt_onset'].iloc[0] if not trial.loc[trial['word'] == trial['adverb'].iloc[0], 'rt_onset'].empty else None
    trial['adverb_onset'] = adverb_onset
    adverb_offset = trial.loc[trial['word'] == trial['adverb'].iloc[0], 'rt_offset'].iloc[0] if not trial.loc[trial['word'] == trial['adverb'].iloc[0], 'rt_offset'].empty else None
    trial['adverb_offset'] = adverb_offset

    verb_onset = trial.loc[trial['word'] == trial['verb'].iloc[0], 'rt_onset'].iloc[0] if not trial.loc[trial['word'] == trial['verb'].iloc[0], 'rt_onset'].empty else None
    trial['verb_onset'] = verb_onset
    verb_offset = trial.loc[trial['word'] == trial['verb'].iloc[0], 'rt_offset'].iloc[0] if not trial.loc[trial['word'] == trial['verb'].iloc[0], 'rt_offset'].empty else None
    trial['verb_offset'] = verb_offset

    by_onset = trial.loc[trial['word'] == 'by', 'rt_onset'].iloc[0] if not trial.loc[trial['word'] == 'by', 'rt_onset'].empty else None
    trial['by_onset'] = by_onset
    by_offset = trial.loc[trial['word'] == 'by', 'rt_offset'].iloc[0] if not trial.loc[trial['word'] == 'by', 'rt_offset'].empty else None
    trial['by_offset'] = by_offset

    pronoun_onset = trial.loc[trial['word'] == trial['pronoun'].iloc[0], 'rt_onset'].iloc[0] if not trial.loc[trial['word'] == trial['pronoun'].iloc[0], 'rt_onset'].empty else None
    trial['pronoun_onset'] = pronoun_onset
    pronoun_offset = trial.loc[trial['word'] == trial['pronoun'].iloc[0], 'rt_offset'].iloc[0] if not trial.loc[trial['word'] == trial['pronoun'].iloc[0], 'rt_offset'].empty else None
    trial['pronoun_offset'] = pronoun_offset

    possessee_onset = trial.loc[trial['word'] == trial['possessee'].iloc[0], 'rt_onset'].iloc[0] if not trial.loc[trial['word'] == trial['possessee'].iloc[0], 'rt_onset'].empty else None
    trial['possessee_onset'] = possessee_onset
    possessee_offset = trial.loc[trial['word'] == trial['possessee'].iloc[0], 'rt_offset'].iloc[0] if not trial.loc[trial['word'] == trial['possessee'].iloc[0], 'rt_offset'].empty else None
    trial['possessee_offset'] = possessee_offset

    return trial

df_crit = df_crit.groupby('filename').apply(find_rt_values).reset_index(drop=True)
df_crit

<ipython-input-48-f869824cf1f9>:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_crit = df_crit.groupby('filename').apply(find_rt_values).reset_index(drop=True)
<ipython-input-48-f869824cf1f9>:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_crit = df_crit.groupby('filename').apply(find_rt_values).reset_index(drop=True)


filename  item           item_type subject_id  \
0        10_feminine_match_auhq  10.0      feminine_match       auhq   
1        10_feminine_match_auhq  10.0      feminine_match       auhq   
2        10_feminine_match_auhq  10.0      feminine_match       auhq   
3        10_feminine_match_auhq  10.0      feminine_match       auhq   
4        10_feminine_match_auhq  10.0      feminine_match       auhq   
...                         ...   ...                 ...        ...   
2800  9_masculine_mismatch_yikz   9.0  masculine_mismatch       yikz   
2801  9_masculine_mismatch_yikz   9.0  masculine_mismatch       yikz   
2802  9_masculine_mismatch_yikz   9.0  masculine_mismatch       yikz   
2803  9_masculine_mismatch_yikz   9.0  masculine_mismatch       yikz   
2804  9_masculine_mismatch_yikz   9.0  masculine_mismatch       yikz   

              word  rt_onset  rt_offset  rt_dur       type condition  ...  \
0        elizabeth    0.6195     1.1695    0.55   feminine     match  ...   
1         suddenly    1.1695     1.5195    0.35   feminine     match  ...   
2        surprised    1.5195     2.0395    0.52   feminine     match  ...   
3              her    2.0395     2.1295    0.09   feminine     match  ...   
4      hairdresser    2.1295     2.6695    0.54   feminine     match  ...   
...            ...       ...        ...     ...        ...       ...  ...   
2800           his    1.8995     2.0595    0.16  masculine  mismatch  ...   
2801  nutritionist    2.0595     2.7695    0.71  masculine  mismatch  ...   
2802            to    2.7695     2.8595    0.09  masculine  mismatch  ...   
2803           the    2.8595     2.9695    0.11  masculine  mismatch  ...   
2804         story    2.9695     3.3095    0.34  masculine  mismatch  ...   

      adverb_onset adverb_offset verb_onset verb_offset by_onset by_offset  \
0           1.1695        1.5195     1.5195      2.0395      NaN       NaN   
1           1.1695        1.5195     1.5195      2.0395      NaN       NaN   
2           1.1695        1.5195     1.5195      2.0395      NaN       NaN   
3           1.1695        1.5195     1.5195      2.0395      NaN       NaN   
4           1.1695        1.5195     1.5195      2.0395      NaN       NaN   
...            ...           ...        ...         ...      ...       ...   
2800        1.0895        1.5195     1.5195      1.8995      NaN       NaN   
2801        1.0895        1.5195     1.5195      1.8995      NaN       NaN   
2802        1.0895        1.5195     1.5195      1.8995      NaN       NaN   
2803        1.0895        1.5195     1.5195      1.8995      NaN       NaN   
2804        1.0895        1.5195     1.5195      1.8995      NaN       NaN   

     pronoun_onset  pronoun_offset  possessee_onset  possessee_offset  
0           2.0395          2.1295           2.1295            2.6695  
1           2.0395          2.1295           2.1295            2.6695  
2           2.0395          2.1295           2.1295            2.6695  
3           2.0395          2.1295           2.1295            2.6695  
4           2.0395          2.1295           2.1295            2.6695  
...            ...             ...              ...               ...  
2800        1.8995          2.0595           2.0595            2.7695  
2801        1.8995          2.0595           2.0595            2.7695  
2802        1.8995          2.0595           2.0595            2.7695  
2803        1.8995          2.0595           2.0595            2.7695  
2804        1.8995          2.0595           2.0595            2.7695  

[2805 rows x 29 columns]

In [ ]:
# add response column
df_crit['response'] = df_crit.groupby('filename')['word'].transform(lambda x: ' '.join(x))
df_crit = df_crit.drop_duplicates()
df_clean = df_crit[['filename','item','subject_id','type','condition','response','possessor_onset','possessor_offset','adverb_onset','adverb_offset','verb_onset','verb_offset','by_onset','by_offset','pronoun_onset','pronoun_offset','possessee_onset','possessee_offset']].drop_duplicates(subset='filename')
df_clean

filename  item subject_id       type condition  \
0        10_feminine_match_auhq  10.0       auhq   feminine     match   
8        10_feminine_match_hbvp  10.0       hbvp   feminine     match   
16       10_feminine_match_kfyi  10.0       kfyi   feminine     match   
23       10_feminine_match_nzne  10.0       nzne   feminine     match   
31    10_feminine_mismatch_lmie  10.0       lmie   feminine  mismatch   
...                         ...   ...        ...        ...       ...   
2764     9_masculine_match_yjml   9.0       yjml  masculine     match   
2772  9_masculine_mismatch_eblb   9.0       eblb  masculine  mismatch   
2780  9_masculine_mismatch_tybj   9.0       tybj  masculine  mismatch   
2788  9_masculine_mismatch_ydiq   9.0       ydiq  masculine  mismatch   
2797  9_masculine_mismatch_yikz   9.0       yikz  masculine  mismatch   

                                               response  possessor_onset  \
0     elizabeth suddenly surprised her hairdresser w...           0.6195   
8     elizabeth generally surprised her hairdresser ...           1.1895   
16      elizabeth suddenly paid her hairdresser a visit           0.5695   
23    elizabeth suddenly surprised her hairdresser w...           0.5995   
31    elizabeth suddenly surprised her plumber with ...           1.6595   
...                                                 ...              ...   
2764    john closely followed his driver into the store           1.4395   
2772  john closely followed his nutritionist into th...           0.7095   
2780  john followed his nutritionist closely to the ...           1.1195   
2788  john closely followed his nutrition nutritioni...           1.3095   
2797  john closely followed his nutritionist to the ...           0.8195   

      possessor_offset  adverb_onset  adverb_offset  verb_onset  verb_offset  \
0               1.1695        1.1695         1.5195      1.5195       2.0395   
8               1.7295           NaN            NaN      2.1895       2.7895   
16              1.1195        1.1195         1.6095         NaN          NaN   
23              1.0295        1.0295         1.4495      1.4495       1.9195   
31              2.2495        2.2495         2.5995      2.5995       3.3695   
...                ...           ...            ...         ...          ...   
2764            1.7995        1.7995         2.2595      2.2595       2.6695   
2772            0.9795        0.9795         1.2995      1.2995       1.6395   
2780            1.4995        3.7195         4.2795      1.4995       1.9795   
2788            1.7295        1.7295         2.2395      2.2395       2.7495   
2797            1.0895        1.0895         1.5195      1.5195       1.8995   

      by_onset  by_offset  pronoun_onset  pronoun_offset  possessee_onset  \
0          NaN        NaN         2.0395          2.1295           2.1295   
8          NaN        NaN         2.7895          2.8595           2.8595   
16         NaN        NaN         1.9495          2.0595           2.0595   
23         NaN        NaN         1.9195          2.0195           2.0195   
31         NaN        NaN         3.4595          3.5595           3.5595   
...        ...        ...            ...             ...              ...   
2764       NaN        NaN         2.6695          2.7795           2.7795   
2772       NaN        NaN         1.6395          1.7995           1.7995   
2780       NaN        NaN         1.9795          2.4095           2.6495   
2788       NaN        NaN         2.7495          2.8795           3.6395   
2797       NaN        NaN         1.8995          2.0595           2.0595   

      possessee_offset  
0               2.6695  
8               3.4595  
16              2.6595  
23              2.6795  
31              3.9495  
...                ...  
2764            3.1495  
2772            2.5795  
2780            3.7195  
2788            4.4795  
2797            2.7695  

[359 rows x 18 columns]

In [ ]:
df_clean.to_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/results_critical_cleaned.csv')

In [ ]:
df_clean.subject_id.nunique()

15

In [ ]:
df_clean.groupby('subject_id').count()

filename  item  type  condition  response  possessor_onset  \
subject_id                                                               
auhq              24    24    24         24        24               24   
eblb              24    24    24         24        24               24   
hbvp              24    24    24         24        24               24   
itng              24    24    24         24        24               22   
jmgf              24    24    24         24        24               24   
kfyi              24    24    24         24        24               22   
lmie              24    24    24         24        24               24   
nbet              24    24    24         24        24               23   
nzne              23    23    23         23        23               22   
rror              24    24    24         24        24               23   
thli              24    24    24         24        24               24   
tybj              24    24    24         24        24               24   
ydiq              24    24    24         24        24               24   
yikz              24    24    24         24        24               24   
yjml              24    24    24         24        24               23   

            possessor_offset  adverb_onset  adverb_offset  verb_onset  \
subject_id                                                              
auhq                      24            24             24          24   
eblb                      24            22             22          24   
hbvp                      24            20             20          22   
itng                      22            22             22          23   
jmgf                      24            23             23          24   
kfyi                      22            19             19          20   
lmie                      24            24             24          24   
nbet                      23            21             21          24   
nzne                      22            17             17          21   
rror                      23            17             17          23   
thli                      24            21             21          18   
tybj                      24            23             23          24   
ydiq                      24            20             20          23   
yikz                      24            23             23          24   
yjml                      23            18             18          21   

            verb_offset  by_onset  by_offset  pronoun_onset  pronoun_offset  \
subject_id                                                                    
auhq                 24         0          0             24              24   
eblb                 24         0          0             24              24   
hbvp                 22         1          1             21              21   
itng                 23         1          1             22              22   
jmgf                 24         0          0             24              24   
kfyi                 20         0          0             19              19   
lmie                 24         0          0             24              24   
nbet                 24         0          0             24              24   
nzne                 21         0          0             21              21   
rror                 23         0          0             17              17   
thli                 18         0          0             16              16   
tybj                 24         0          0             24              24   
ydiq                 23         1          1             16              16   
yikz                 24         0          0             23              23   
yjml                 21         0          0             14              14   

            possessee_onset  possessee_offset  
subject_id                                     
auhq                     24                24  
eblb                   

## SV AGREE

In [ ]:
df_svagree_results = df_results_id[(df_results_id.item > 24)&(df_results_id.item < 57)]
df_svagree_results.item = df_svagree_results.item.astype(int)
df_svagree_results['condition'] = df_svagree_results.type
df_svagree_results

<ipython-input-60-8fd4e9131770>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_svagree_results.item = df_svagree_results.item.astype(int)
<ipython-input-60-8fd4e9131770>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_svagree_results['condition'] = df_svagree_results.type


filename  item item_type subject_id           word  rt_onset  \
167   26_PP_yjml    26        PP       yjml            the    1.2595   
168   26_PP_yjml    26        PP       yjml  announcements    1.4095   
169   26_PP_yjml    26        PP       yjml             by    2.1995   
170   26_PP_yjml    26        PP       yjml            the    2.5795   
171   26_PP_yjml    26        PP       yjml      directors    2.6995   
...          ...   ...       ...        ...            ...       ...   
8734  56_SS_nzne    56        SS       nzne            the    1.6695   
8735  56_SS_nzne    56        SS       nzne         bridge    1.7595   
8736  56_SS_nzne    56        SS       nzne            was    2.0195   
8737  56_SS_nzne    56        SS       nzne     undeniably    2.1595   
8738  56_SS_nzne    56        SS       nzne          noisy    2.8795   

      rt_offset  rt_dur type condition  word_order     time_id  
167      1.4095  0.1500   PP        PP         1.0  1727711888  
168      2.0995  0.6900   PP        PP         2.0  1727711888  
169      2.5795  0.3800   PP        PP         3.0  1727711888  
170      2.6995  0.1200   PP        PP         4.0  1727711888  
171      3.3495  0.6500   PP        PP         5.0  1727711888  
...         ...     ...  ...       ...         ...         ...  
8734     1.7595  0.0900   SS        SS         4.0  1727727799  
8735     2.0195  0.2600   SS        SS         5.0  1727727799  
8736     2.1595  0.1400   SS        SS         6.0  1727727799  
8737     2.8795  0.7200   SS        SS         7.0  1727727799  
8738     3.4200  0.5405   SS        SS         8.0  1727727799  

[3673 rows x 12 columns]

In [ ]:
stim_svagree = pd.read_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/exp3_stimuli_svagree_real.csv')
stim_svagree.item = stim_svagree.item.astype(int)
df_svagree = pd.merge(df_svagree_results,stim_svagree[['item','condition','sentence_target']],how='left',on=['item','condition'])
df_svagree = df_svagree.drop(columns=['item_type','type'])
df_svagree

filename  item subject_id           word  rt_onset  rt_offset  rt_dur  \
0     26_PP_yjml    26       yjml            the    1.2595     1.4095  0.1500   
1     26_PP_yjml    26       yjml  announcements    1.4095     2.0995  0.6900   
2     26_PP_yjml    26       yjml             by    2.1995     2.5795  0.3800   
3     26_PP_yjml    26       yjml            the    2.5795     2.6995  0.1200   
4     26_PP_yjml    26       yjml      directors    2.6995     3.3495  0.6500   
...          ...   ...        ...            ...       ...        ...     ...   
3668  56_SS_nzne    56       nzne            the    1.6695     1.7595  0.0900   
3669  56_SS_nzne    56       nzne         bridge    1.7595     2.0195  0.2600   
3670  56_SS_nzne    56       nzne            was    2.0195     2.1595  0.1400   
3671  56_SS_nzne    56       nzne     undeniably    2.1595     2.8795  0.7200   
3672  56_SS_nzne    56       nzne          noisy    2.8795     3.4200  0.5405   

     condition  word_order     time_id  \
0           PP         1.0  1727711888   
1           PP         2.0  1727711888   
2           PP         3.0  1727711888   
3           PP         4.0  1727711888   
4           PP         5.0  1727711888   
...        ...         ...         ...   
3668        SS         4.0  1727727799   
3669        SS         5.0  1727727799   
3670        SS         6.0  1727727799   
3671        SS         7.0  1727727799   
3672        SS         8.0  1727727799   

                                        sentence_target  
0     The announcements by the directors certainly w...  
1     The announcements by the directors certainly w...  
2     The announcements by the directors certainly w...  
3     The announcements by the directors certainly w...  
4     The announcements by the directors certainly w...  
...                                                 ...  
3668      The truck on the bridge undeniably was noisy.  
3669      The truck on the bridge undeniably was noisy.  
3670      The truck on the bridge undeniably was noisy.  
3671      The truck on the bridge undeniably was noisy.  
3672      The truck on the bridge undeniably was noisy.  

[3673 rows x 11 columns]

In [ ]:
def get_attractor(sentence):
    words = sentence.split()
    if 'was' in words:
        index = words.index('was')
    elif 'were' in words:
        index = words.index('were')
    else:
        return None
    if index > 0:
        return words[index - 2]
    return None
def get_adverb(sentence):
    words = sentence.split()
    if 'was' in words:
        index = words.index('was')
    elif 'were' in words:
        index = words.index('were')
    else:
        return None
    if index > 0:
        return words[index - 1]
    return None
df_svagree['head'] = df_svagree['sentence_target'].apply(lambda x: x.split()[1])
df_svagree['attractor'] = df_svagree['sentence_target'].apply(get_attractor)
df_svagree['adverb'] = df_svagree['sentence_target'].apply(get_adverb)
df_svagree['verb'] = df_svagree['sentence_target'].apply(lambda x: 'was' if 'was' in x else ('were' if 'were' in x else None))
df_svagree

filename  item subject_id           word  rt_onset  rt_offset  rt_dur  \
0     26_PP_yjml    26       yjml            the    1.2595     1.4095  0.1500   
1     26_PP_yjml    26       yjml  announcements    1.4095     2.0995  0.6900   
2     26_PP_yjml    26       yjml             by    2.1995     2.5795  0.3800   
3     26_PP_yjml    26       yjml            the    2.5795     2.6995  0.1200   
4     26_PP_yjml    26       yjml      directors    2.6995     3.3495  0.6500   
...          ...   ...        ...            ...       ...        ...     ...   
3668  56_SS_nzne    56       nzne            the    1.6695     1.7595  0.0900   
3669  56_SS_nzne    56       nzne         bridge    1.7595     2.0195  0.2600   
3670  56_SS_nzne    56       nzne            was    2.0195     2.1595  0.1400   
3671  56_SS_nzne    56       nzne     undeniably    2.1595     2.8795  0.7200   
3672  56_SS_nzne    56       nzne          noisy    2.8795     3.4200  0.5405   

     condition  word_order     time_id  \
0           PP         1.0  1727711888   
1           PP         2.0  1727711888   
2           PP         3.0  1727711888   
3           PP         4.0  1727711888   
4           PP         5.0  1727711888   
...        ...         ...         ...   
3668        SS         4.0  1727727799   
3669        SS         5.0  1727727799   
3670        SS         6.0  1727727799   
3671        SS         7.0  1727727799   
3672        SS         8.0  1727727799   

                                        sentence_target           head  \
0     The announcements by the directors certainly w...  announcements   
1     The announcements by the directors certainly w...  announcements   
2     The announcements by the directors certainly w...  announcements   
3     The announcements by the directors certainly w...  announcements   
4     The announcements by the directors certainly w...  announcements   
...                                                 ...            ...   
3668      The truck on the bridge undeniably was noisy.          truck   
3669      The truck on the bridge undeniably was noisy.          truck   
3670      The truck on the bridge undeniably was noisy.          truck   
3671      The truck on the bridge undeniably was noisy.          truck   
3672      The truck on the bridge undeniably was noisy.          truck   

      attractor      adverb  verb  
0     directors   certainly  were  
1     directors   certainly  were  
2     directors   certainly  were  
3     directors   certainly  were  
4     directors   certainly  were  
...         ...         ...   ...  
3668     bridge  undeniably   was  
3669     bridge  undeniably   was  
3670     bridge  undeniably   was  
3671     bridge  undeniably   was  
3672     bridge  undeniably   was  

[3673 rows x 15 columns]

In [ ]:
def find_rt_values(trial):

    head_onset = trial.loc[trial['word'] == trial['head'].iloc[0], 'rt_onset'].iloc[0] if not trial.loc[trial['word'] == trial['head'].iloc[0], 'rt_onset'].empty else None
    trial['head_onset'] = head_onset
    head_offset = trial.loc[trial['word'] == trial['head'].iloc[0], 'rt_offset'].iloc[0] if not trial.loc[trial['word'] == trial['head'].iloc[0], 'rt_offset'].empty else None
    trial['head_offset'] = head_offset

    attractor_onset = trial.loc[trial['word'] == trial['attractor'].iloc[0], 'rt_onset'].iloc[0] if not trial.loc[trial['word'] == trial['attractor'].iloc[0], 'rt_onset'].empty else None
    trial['attractor_onset'] = attractor_onset
    attractor_offset = trial.loc[trial['word'] == trial['attractor'].iloc[0], 'rt_offset'].iloc[0] if not trial.loc[trial['word'] == trial['attractor'].iloc[0], 'rt_offset'].empty else None
    trial['attractor_offset'] = attractor_offset

    adverb_onset = trial.loc[trial['word'] == trial['adverb'].iloc[0], 'rt_onset'].iloc[0] if not trial.loc[trial['word'] == trial['adverb'].iloc[0], 'rt_onset'].empty else None
    trial['adverb_onset'] = adverb_onset
    adverb_offset = trial.loc[trial['word'] == trial['adverb'].iloc[0], 'rt_offset'].iloc[0] if not trial.loc[trial['word'] == trial['adverb'].iloc[0], 'rt_offset'].empty else None
    trial['adverb_offset'] = adverb_offset

    verb_onset = trial.loc[trial['word'] == trial['verb'].iloc[0], 'rt_onset'].iloc[0] if not trial.loc[trial['word'] == trial['verb'].iloc[0], 'rt_onset'].empty else None
    trial['verb_onset'] = verb_onset
    verb_offset = trial.loc[trial['word'] == trial['verb'].iloc[0], 'rt_offset'].iloc[0] if not trial.loc[trial['word'] == trial['verb'].iloc[0], 'rt_offset'].empty else None
    trial['verb_offset'] = verb_offset

    return trial

df_svagree = df_svagree.groupby('filename').apply(find_rt_values).reset_index(drop=True)
df_svagree

<ipython-input-65-8cae98f8e80d>:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_svagree = df_svagree.groupby('filename').apply(find_rt_values).reset_index(drop=True)
<ipython-input-65-8cae98f8e80d>:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_svagree = df_svagree.groupby('filename').apply(find_rt_values).reset_index(drop=True)


filename  item subject_id            word  rt_onset  rt_offset  \
0     25_PP_auhq    25       auhq             the    0.5695     0.6095   
1     25_PP_auhq    25       auhq  advertisements    0.6095     1.1895   
2     25_PP_auhq    25       auhq            from    1.1895     1.3095   
3     25_PP_auhq    25       auhq             the    1.3095     1.4395   
4     25_PP_auhq    25       auhq           clubs    1.4395     1.7195   
...          ...   ...        ...             ...       ...        ...   
3668  56_SS_nzne    56       nzne             the    1.6695     1.7595   
3669  56_SS_nzne    56       nzne          bridge    1.7595     2.0195   
3670  56_SS_nzne    56       nzne             was    2.0195     2.1595   
3671  56_SS_nzne    56       nzne      undeniably    2.1595     2.8795   
3672  56_SS_nzne    56       nzne           noisy    2.8795     3.4200   

      rt_dur condition  word_order     time_id  ...      adverb  verb  \
0     0.0400        PP         1.0  1727727259  ...   obviously  were   
1     0.5800        PP         2.0  1727727259  ...   obviously  were   
2     0.1200        PP         3.0  1727727259  ...   obviously  were   
3     0.1300        PP         4.0  1727727259  ...   obviously  were   
4     0.2800        PP         5.0  1727727259  ...   obviously  were   
...      ...       ...         ...         ...  ...         ...   ...   
3668  0.0900        SS         4.0  1727727799  ...  undeniably   was   
3669  0.2600        SS         5.0  1727727799  ...  undeniably   was   
3670  0.1400        SS         6.0  1727727799  ...  undeniably   was   
3671  0.7200        SS         7.0  1727727799  ...  undeniably   was   
3672  0.5405        SS         8.0  1727727799  ...  undeniably   was   

     head_onset head_offset attractor_onset  attractor_offset  adverb_onset  \
0        0.6095      1.1895          1.4395            1.7195        1.7895   
1        0.6095      1.1895          1.4395            1.7195        1.7895   
2        0.6095      1.1895          1.4395            1.7195        1.7895   
3        0.6095      1.1895          1.4395            1.7195        1.7895   
4        0.6095      1.1895          1.4395            1.7195        1.7895   
...         ...         ...             ...               ...           ...   
3668     1.2195      1.4995          1.7595            2.0195        2.1595   
3669     1.2195      1.4995          1.7595            2.0195        2.1595   
3670     1.2195      1.4995          1.7595            2.0195        2.1595   
3671     1.2195      1.4995          1.7595            2.0195        2.1595   
3672     1.2195      1.4995          1.7595            2.0195        2.1595   

      adverb_offset  verb_onset  verb_offset  
0            2.2395      2.2395       2.3895  
1            2.2395      2.2395       2.3895  
2            2.2395      2.2395       2.3895  
3            2.2395      2.2395       2.3895  
4            2.2395      2.2395       2.3895  
...             ...         ...          ...  
3668         2.8795      2.0195       2.1595  
3669         2.8795      2.0195       2.1595  
3670         2.8795      2.0195       2.1595  
3671         2.8795      2.0195       2.1595  
3672         2.8795      2.0195       2.1595  

[3673 rows x 23 columns]

In [ ]:
# add response column
df_svagree['response'] = df_svagree.groupby('filename')['word'].transform(lambda x: ' '.join(x))
df_svagree = df_svagree.drop_duplicates()
df_svagree_clean = df_svagree[['filename','item','subject_id','condition','response','head_onset','head_offset','attractor_onset','attractor_offset','adverb_onset','adverb_offset','verb_onset','verb_offset']].drop_duplicates(subset='filename')
df_svagree_clean

filename  item subject_id condition  \
0     25_PP_auhq    25       auhq        PP   
8     25_PP_hbvp    25       hbvp        PP   
16    25_PP_kfyi    25       kfyi        PP   
24    25_PP_nzne    25       nzne        PP   
28    25_PS_itng    25       itng        PS   
...          ...   ...        ...       ...   
3633  56_SP_yjml    56       yjml        SP   
3641  56_SS_auhq    56       auhq        SS   
3649  56_SS_hbvp    56       hbvp        SS   
3657  56_SS_kfyi    56       kfyi        SS   
3665  56_SS_nzne    56       nzne        SS   

                                               response  head_onset  \
0     the advertisements from the clubs obviously we...      0.6095   
8     the advertisements from the clubs were obvious...      0.6095   
16    the advertisements of the club were probably c...      1.1195   
24                      the advertisements are colorful      0.8695   
28    the advertisement for the club was obviously c...         NaN   
...                                                 ...         ...   
3633      the trucks on the bridge are undeniably noisy         NaN   
3641       the truck on the bridge undeniably was noisy      0.6695   
3649       the truck on the bridge was undeniably noisy      0.9095   
3657        the truck on the bridge was undeniably busy      1.6095   
3665       the truck on the bridge was undeniably noisy      1.2195   

      head_offset  attractor_onset  attractor_offset  adverb_onset  \
0          1.1895           1.4395            1.7195        1.7895   
8          1.2495           1.4595            1.8295        1.9695   
16         1.8895              NaN               NaN           NaN   
24         1.7395              NaN               NaN           NaN   
28            NaN           4.8495            5.1695        5.2995   
...           ...              ...               ...           ...   
3633          NaN              NaN               NaN        2.1995   
3641       0.7695           0.9595            1.2495        1.2495   
3649       1.1695           1.3495            1.6895        1.8195   
3657       2.0695           3.0195            3.3895        4.0595   
3665       1.4995           1.7595            2.0195        2.1595   

      adverb_offset  verb_onset  verb_offset  
0            2.2395      2.2395       2.3895  
8            2.4195      1.8295       1.9695  
16              NaN      2.6695       2.8295  
24              NaN         NaN          NaN  
28           5.8695         NaN          NaN  
...             ...         ...          ...  
3633         2.9095         NaN          NaN  
3641         1.9495      1.9495       2.1795  
3649         2.4295      1.6895       1.8195  
3657         5.1095      3.3895       3.9795  
3665         2.8795      2.0195       2.1595  

[476 rows x 13 columns]

In [ ]:
df_svagree_clean.to_csv(f'/content/drive/MyDrive/{exp_label}/{exp_spec}/results_svagree_cleaned.csv')

In [ ]:
df_svagree_clean.subject_id.nunique()

15

In [ ]:
df_svagree_clean.groupby('subject_id').count()

filename  item  condition  response  head_onset  head_offset  \
subject_id                                                                 
auhq              32    32         32        32          32           32   
eblb              32    32         32        32          31           31   
hbvp              32    32         32        32          29           29   
itng              31    31         31        31          22           22   
jmgf              31    31         31        31          26           26   
kfyi              32    32         32        32          20           20   
lmie              32    32         32        32          30           30   
nbet              32    32         32        32          27           27   
nzne              32    32         32        32          20           20   
rror              30    30         30        30          24           24   
thli              32    32         32        32          29           29   
tybj              32    32         32        32          28           28   
ydiq              32    32         32        32          27           27   
yikz              32    32         32        32          29           29   
yjml              32    32         32        32          23           23   

            attractor_onset  attractor_offset  adverb_onset  adverb_offset  \
subject_id                                                                   
auhq                     30                30            32             32   
eblb                     28                28            32             32   
hbvp                     27                27            30             30   
itng                     15                15            23             23   
jmgf                     21                21            30             30   
kfyi                     23                23            19             19   
lmie                     24                24            26             26   
nbet                     17                17            18             18   
nzne                     18                18            19             19   
rror                     26                26            19             19   
thli                     24                24            27             27   
tybj                     26                26            31             31   
ydiq                     23                23            24             24   
yikz                     27                27            28             28   
yjml                     22                22            30             30   

            verb_onset  verb_offset  
subject_id                           
auhq                32           32  
eblb                26           26  
hbvp                27           27  
itng                15           15  
jmgf                24           24  
kfyi                18           18  
lmie                31           31  
nbet                26           26  
nzne                 6            6  
rror                26           26  
thli                28           28  
tybj                21           21  
ydiq                25           25  
yikz                27           27  
yjml                 4            4